In [13]:
# # Example for Google Colab with CUDA 11
# !pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 119.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
import pandas as pd

In [6]:
df = pd.read_csv("../results/attention_metrics_20251222_092224.csv", usecols=["sample_id","layer","head","token_pos", "query_norm","entropy","max_attn","k_eff"])

In [7]:
df.head()

,sample_id,layer,head,token_pos,query_norm,entropy,max_attn,k_eff
0,0,0,0,0,19.30,NaN,1.000000,1.0
1,0,0,0,1,20.08,NaN,0.510742,2.0
2,0,0,0,2,23.78,0.959532,0.587891,3.0
3,0,0,0,3,20.88,1.322193,0.378906,4.0
4,0,0,0,4,17.88,1.499800,0.364990,4.0


In [16]:
df.describe()

,sample_id,layer,head,token_pos,query_norm,entropy,max_attn,k_eff
count,3.342336e+07,3.342336e+07,3.342336e+07,3.342336e+07,3.342336e+07,3.342336e+07,3.342336e+07,3.342336e+07
mean,3.150000e+01,1.550000e+01,1.550000e+01,2.565000e+02,1.342773e+01,3.303503e+00,3.967942e-01,1.049264e+02
std,1.847295e+01,9.233093e+00,9.233093e+00,1.472240e+02,2.314435e+00,1.656458e+00,3.112716e-01,9.191588e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,8.687000e-01,-0.000000e+00,3.187180e-03,1.000000e+00
25%,1.575000e+01,7.750000e+00,7.750000e+00,1.290000e+02,1.192000e+01,1.914854e+00,1.075439e-01,2.400000e+01
50%,3.150000e+01,1.550000e+01,1.550000e+01,2.565000e+02,1.323400e+01,3.555413e+00,3.205566e-01,8.300000e+01
75%,4.725000e+01,2.325000e+01,2.325000e+01,3.840000e+02,1.469500e+01,4.741777e+00,6.738281e-01,1.690000e+02
max,6.300000e+01,3.100000e+01,3.100000e+01,5.110000e+02,5.372000e+01,6.217727e+00,1.000000e+00,4.450000e+02


In [17]:
len(df)

33423360

In [10]:
# Drop invalid entropy rows
df = df[df["entropy"].notna()]

# Drop first token (degenerate attention)
df = df[df["token_pos"] > 0]

In [18]:
import numpy as np
from scipy.stats import pearsonr

def summarize_head(group):
    q = group["query_norm"].values
    h = group["entropy"].values
    a = group["max_attn"].values
    k = group["k_eff"].values

    r_q_h, p_q_h = pearsonr(q, h)
    r_q_a, _ = pearsonr(q, a)
    r_q_k, _ = pearsonr(q, k)

    return pd.Series({
        "mean_q_norm": q.mean(),
        "mean_entropy": h.mean(),
        "mean_max_attn": a.mean(),
        "mean_k_eff": k.mean(),
        "r_q_entropy": r_q_h,
        "p_q_entropy": p_q_h,
        "r_q_max_attn": r_q_a,
        "r_q_k_eff": r_q_k,
        "num_tokens": len(q)
    })

summary = (
    df
    .groupby(["sample_id","layer","head"], sort=False)
    .apply(summarize_head)
    .reset_index()
)

/tmp/ipykernel_98917/2604092697.py:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r_q_k, _ = pearsonr(q, k)
/tmp/ipykernel_98917/2604092697.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_head)


In [19]:
summary.to_csv("../results/head_summary.csv", index=False)

In [20]:
!pwd

/home/sagemaker-user/Spectra/scripts
